In [ ]:
# https://github.com/WackAlley/worksheet/tree/main/labcourse21

In [137]:
import numpy as np
from icecream import ic
import pandas as pd

## Kinetische Impulsbestimmung

Skalierungsfaktor $s_P$
\begin{align}
    s_P &= \frac{L_{1}}{30.01cm} - \frac{1}{30.01cm}\frac{L_{1} - L_{2}}{2} \\
    \Delta s_{P} &= \sqrt{ \left(\frac{\partial s_{P}}{\partial L_{1}} \Delta L_{1}\right)^{2} + 
                           \left(\frac{\partial s_{P}}{\partial L_{2}} \Delta L_{2}\right)^{2}}
\end{align}

Krümmungsradius $r_{real}$ in $cm$
\begin{align}
    r_{real} &= \frac{r_{gemessen}}{s_{P}} \\
    \Delta r_{real} &= \sqrt{ \left( \frac{\partial r_{real}}{\partial r_{gem}} \Delta r_{gem} \right)^2 + \left( \frac{\partial r_{real}}{\partial s_{P}} \Delta s_{P} \right)^2 } \\
                    &= \sqrt{\left(\frac{\Delta r_{gem}}{s_{P}}\right)^2 + \left( \frac{r_{gem}}{s_{P}^2} \Delta s_{P} \right)^2}
\end{align}

Impuls $p$ in $\frac{MeV}{c}$
\begin{align}
    p & = 2.998 \cdot B[T] \cdot r_{real}[cm] \frac{MeV}{c\cdot cm \cdot T} \\
    \Delta p &= \sqrt{ \left( \frac{\partial p}{\partial r_{real}} \Delta r_{real} \right)^2 } \\
             &= \sqrt{2.998 \cdot 2.05 \cdot \Delta r_{real}}
\end{align}

In [138]:
# Positionen, Einheit in cm
l1 = 45.5 
delta_l1 = 0.1
l2 = 59
delta_l2 = 0.1

# Skalierungsfaktor s_p
s_p = l1 - (l1 - l2)/2
s_p = s_p/30.01

ic(s_p)

# Fehler Skalierungsfaktor
delta_s_p_inner_term = (delta_l1 / (2*30.01))**2 + (delta_l2 / (2*30.01))**2
delta_s_p = np.sqrt(delta_s_p_inner_term)
ic(delta_s_p)

# Krümmungsradius
def r_real(r_gem): 
    return r_gem / s_p # Krümmungsradius r_real in cm

# Fehler Krümmungsradius
def delta_r_real(r_gem, delta_r_gem):
    delta_r_real_inner_term = (delta_r_gem / s_p)**2 + ( (r_gem / s_p**2) * delta_s_p)**2
    delta_r_real_complete_term = np.sqrt(delta_r_real_inner_term)
    return delta_r_real_complete_term

# Impuls, kinetischer Ausdruck in MeV/c
B = 2.05 # Magnetische Flussstärke in T
def momentum(r_real):
    return 2.998 * B * r_real


# Fehler Impuls, kinetischer Ausdruck
def delta_momentum(delta_r_real):
    return 2.998 * B * delta_r_real

# Alles in einem
dfs = [] # Gesamtdataframe für alle Topologien
def parameterset(r_gem, delta_r_gem):
    
    # ic(r_real(r_gem))
    # ic(delta_r_real(r_gem, delta_r_gem))
    # ic(momentum(r_real(r_gem)))
    # ic(delta_momentum(delta_r_real(r_gem, delta_r_gem)))

    # Array
    r_gem_array = np.atleast_1d(r_gem)  
    delta_r_gem_array = np.atleast_1d(delta_r_gem)  

    # Tabellierung
    # Um nur die Tabelle aufzurufen, gebe ein: parameterset(r_gem, delta_r_gem)[4]
    r_real_values = r_real(r_gem_array)
    delta_r_real_values = delta_r_real(r_gem_array, delta_r_gem_array)
    momentum_values = momentum(r_real_values)
    delta_momentum_values = delta_momentum(delta_r_real_values)
    
    df = pd.DataFrame({
        "r_gem": r_gem_array,
        "delta_r_gem": delta_r_gem_array,
        "r_real": r_real_values,
        "delta_r_real": delta_r_real_values,
        "momentum": momentum_values,
        "delta_momentum": delta_momentum_values
    }, index=np.arange(1, len(r_gem_array) + 1))

    dfs.append(df)
    
    return r_real(r_gem), delta_r_real(r_gem, delta_r_gem), momentum(r_real(r_gem)), delta_momentum(delta_r_real(r_gem, delta_r_gem)), df

ic| s_p: 1.7410863045651448
ic| delta_s_p: 0.0023562371915579724


In [139]:
# Maßstabsetzung, Impuls = 810MeV/c
r_gem1 = 230 # gemessener Krümmungsradius in cm
delta_r_gem1 = 7.5

df1 = parameterset(r_gem1, delta_r_gem1)[4]
df1

,r_gem,delta_r_gem,r_real,delta_r_real,momentum,delta_momentum
1,230,7.5,132.101435,4.311364,811.882212,26.49721


In [140]:
# Folie 223
r_gem1 = 65
delta_r_gem1 = 7.5

r_gem2 = 80 
delta_r_gem2 = 7.5

r_gem3 = 90
delta_r_gem3 = 7.5

df2 = parameterset(np.array([r_gem1,r_gem2,r_gem3]),np.array([delta_r_gem1,delta_r_gem2,delta_r_gem3,]))[4]
df2

,r_gem,delta_r_gem,r_real,delta_r_real,momentum,delta_momentum
1,65,7.5,37.333014,4.307952,229.444973,26.476241
2,80,7.5,45.948325,4.308104,282.393813,26.477178
3,90,7.5,51.691866,4.308223,317.693039,26.477911


In [141]:
# Folie 229, Topologie 201
r_gem1 = 140
delta_r_gem1 = 4

r_gem2 = 36
delta_r_gem2 = 2

r_gem3 = 42.5
delta_r_gem3 = 2

r_gem4 = 33
delta_r_gem4 = 2

df3 = parameterset(np.array([r_gem1,r_gem2,r_gem3,r_gem4]),np.array([delta_r_gem1,delta_r_gem2,delta_r_gem3,delta_r_gem4]))[4]
df3

,r_gem,delta_r_gem,r_real,delta_r_real,momentum,delta_momentum
1,140.0,4,80.409569,2.299992,494.189172,14.135521
2,36.0,2,20.676746,1.149049,127.077216,7.061940
3,42.5,2,24.410048,1.149183,150.021713,7.062764
4,33.0,2,18.953684,1.148994,116.487448,7.061605


In [142]:
# Folie 239
r_gem1 = 113
delta_r_gem1 = 5

r_gem2 = 215
delta_r_gem2 = 6

df4 = parameterset(np.array([r_gem1,r_gem2]),np.array([delta_r_gem1,delta_r_gem2]))[4]
df4 

,r_gem,delta_r_gem,r_real,delta_r_real,momentum,delta_momentum
1,113,5,64.902010,2.873113,398.881261,17.657866
2,215,6,123.486124,3.450174,758.933372,21.204425


In [143]:
# Folie 285
r_gem1 = 54
delta_r_gem1 = 2

df5 = parameterset(np.array([r_gem1]),np.array([delta_r_gem1]))[4]
df5

,r_gem,delta_r_gem,r_real,delta_r_real,momentum,delta_momentum
1,54,2,31.01512,1.149475,190.615824,7.064557


In [145]:
# Gesamtdataframe dfs

k=0
firstindex = []
secondindex = []
for df in dfs:
    indexlength = len(df.index)
    # firstindexentry = (np.ones(indexlength) + k).tolist() 
    firstindexentry = (np.ones(indexlength) + k).astype(int).tolist()
    firstindex.extend(firstindexentry) 
    j=0
    for j in range(1, indexlength + 1):
        secondindex.append(j)
    k += 1


total_df = pd.concat(dfs, axis=0)
total_df.index = pd.MultiIndex.from_arrays([firstindex, secondindex], names=["Versuch", "Teilchen"])
total_df.columns = [f'r$_{{gem}} [cm]$', f'$\\Delta r_{{gem}} [cm]$', f'$r_{{real}} [cm]$', f'$\\Delta r_{{real}} [cm]$', f'$p [\\frac{{MeV}}{{c}}]$', f'$\\Delta p [\\frac{{MeV}}{{c}}]$']

total_df

# latex_table = total_df.to_latex(
#     index=True, 
#     escape=False,
#     caption="Parameterwerte für die Impulsberechnung mittels Kinematik. 1 und 2: Topologie 300, 3: Topologie 201, 4: Topologie 200, 5: Topologie 100",
#     label="tab:momentumkinematic", 
#     position="H",  
#     column_format="llrrrrrr", 
#     float_format="{:0.4f}".format)
# print(latex_table)

r$_{gem} [cm]$  $\Delta r_{gem} [cm]$  $r_{real} [cm]$  \
Versuch Teilchen                                                           
1       1                  230.0                    7.5       132.101435   
2       1                   65.0                    7.5        37.333014   
        2                   80.0                    7.5        45.948325   
        3                   90.0                    7.5        51.691866   
3       1                  140.0                    4.0        80.409569   
        2                   36.0                    2.0        20.676746   
        3                   42.5                    2.0        24.410048   
        4                   33.0                    2.0        18.953684   
4       1                  113.0                    5.0        64.902010   
        2                  215.0                    6.0       123.486124   
5       1                   54.0                    2.0        31.015120   

                  $\Delta r_{real} [cm]$  $p [\frac{MeV}{c}]$  \
Versuch Teilchen                                                
1       1                       4.311364           811.882212   
2       1                       4.307952           229.444973   
        2                       4.308104           282.393813   
        3                       4.308223           317.693039   
3       1                       2.299992           494.189172   
        2                       1.149049           127.077216   
        3                       1.149183           150.021713   
        4                       1.148994           116.487448   
4       1                       2.873113           398.881261   
        2                       3.450174           758.933372   
5       1                       1.149475           190.615824   

                  $\Delta p [\frac{MeV}{c}]$  
Versuch Teilchen                              
1       1                          26.497210  
2       1                          26.476241  
        2                          26.477178  
        3                          26.477911  
3       1                          14.135521  
        2                           7.061940  
        3                           7.062764  
        4                           7.061605  
4       1                          17.657866  
        2                          21.204425  
5       1                           7.064557